# Анализ факторов, влияющих на успеваемость студентов

## Подготовка к работе

In [14]:
# Импорт необходимых для работы библиотек

import pandas as pd
from scipy import stats as st

In [15]:
# Отключение предупреждений в итоговом отчёте

import warnings
warnings.filterwarnings('ignore')

In [16]:
# Выгрузка предобработанной версии датасета `jamb`

jamb = pd.read_csv('jamb.csv')

In [17]:
jamb.head()

,jamb_score,study_hours_per_week,attendance_rate,teacher_quality,distance_to_school,school_type,school_location,extra_tutorials,access_to_learning_materials,parent_involvement,it_knowledge,age,gender,socioeconomic_status,parent_education_level,assignments_completed
0,192,22,78,4,12.4,Public,Urban,Yes,Yes,High,Medium,17,Male,Low,Tertiary,2
1,207,14,88,4,2.7,Public,Rural,No,Yes,High,High,15,Male,High,Unknown,1
2,182,29,87,2,9.6,Public,Rural,Yes,Yes,High,Medium,20,Female,High,Tertiary,2
3,210,29,99,2,2.6,Public,Urban,No,Yes,Medium,High,22,Female,Medium,Tertiary,1
4,199,12,98,3,8.8,Public,Urban,No,Yes,Medium,Medium,22,Female,Medium,Tertiary,1


## Проверка гипотез

### Верно ли, что студенты, проживающие ближе к месту обучения, сдают экзамены лучше?

**Предпосылки:**  студенты, которые живут далеко от школы, начинают свой день раньше остальных. Не исключено, что из-за этого они меньше спят и соответственно меньше отдыхают. Утренняя дорога - это нагрузка на организм, которая приводит к снижению уровня энергии, с которой студент начинает учебный день. Из-за этого он может быть менее сконцентрирован и вовлечен в процесс, а значит может и хуже усваивать материал.

Медианное значение расстояния равно 10.0 км, то есть ровно половина студентов проживает на расстоянии менее 10 км от школы, а ровно половина – более. Сделаем срезы на основании этого значения:

In [18]:
distance_less = jamb[jamb['distance_to_school'] <= 10]
distance_more = jamb[jamb['distance_to_school'] > 10]

Чтобы сформулировать гипотезы и выбрать нужный тест, проверим данные на нормальность с помощью теста Шапиро-Уилка:

In [19]:
# Тест Шапиро-Уилка

shapiro_less = st.shapiro(distance_less['jamb_score'])
print(f"Для расстояния <=10 км: statistic = {shapiro_less.statistic}, p-value = {shapiro_less.pvalue}")

shapiro_more = st.shapiro(distance_more['jamb_score'])
print(f"Для расстояния >10 км:statistic = {shapiro_more.statistic}, p-value = {shapiro_more.pvalue}")

alpha = 0.05

if shapiro_less.pvalue < alpha:
    print("Данные в группе c расстоянием не более 10 км до школы не распределены нормально")
else:
    print("Данные в группе c расстоянием не более 10 км до школы распределены нормально")

if shapiro_more.pvalue < alpha:
    print("Данные в группе c расстоянием более 10 км до школы не распределены нормально")
else:
    print("Данные в группе c расстоянием более 10 км до школы распределены нормально")

Для расстояния <=10 км: statistic = 0.9704512878956744, p-value = 1.7251723901474076e-22
Для расстояния >10 км:statistic = 0.9645573587805363, p-value = 4.213763940127312e-24
Данные в группе c расстоянием не более 10 км до школы не распределены нормально
Данные в группе c расстоянием более 10 км до школы не распределены нормально


Видим, что данные обеих групп не распределены нормально, поэтому использовать t-тест нежелательно.

В таком случае воспользуемся **тестом Манна-Уитни** для сравнения двух независимых групп. Сформулируем гипотезы:

**Нулевая гипотеза Н0:** распределения баллов JAMB у студентов, проживающих ≤10 км и >10 км от школы, одинаковы.

**Альтернативная правосторонняя гипотеза Н1**: у студентов, которые живут ближе к школе (≤10 км), распределение баллов за экзамен смещено в сторону более высоких значений по сравнению с распределением баллов у студентов, проживающих >10 км от школы.

In [20]:
# Тест Манна-Уитни для двух независимых выборок

alpha = 0.05

result = st.mannwhitneyu(distance_less['jamb_score'], distance_more['jamb_score'], alternative='greater')

print(f"U-статистика: {result.statistic:.2f}, p-значение: {result.pvalue}")

if result.pvalue < alpha:
      print('Отвергаем нулевую гипотезу')
else:
      print('Не получилось отвергнуть нулевую гипотезу')

U-статистика: 3390343.50, p-значение: 8.99400123779393e-08
Отвергаем нулевую гипотезу


📌 **Выводы:**
* Результаты теста Манна-Уитни показали статистически значимое различие между баллами JAMB студентов, проживающих на расстоянии менее или равном 10 км от школы, и студентов, проживающих дальше. Р-значение значительно меньше уровня значимости, поэтому можно отвергнуть нулевую гипотезу. Баллы студентов, живущих ближе к школе, статистически выше, чем баллы студентов, живущих на большем расстоянии.

##Проверка Гипотез (Артём)

###**Верно ли, что студенты, обучающиеся в частных школах, имеют более высокие шансы сдать экзамен успешно?**


**Предпосылки:**

Представьте, что вы член приёмной коммисии. Вы хотите отобрать себе самых умных студентов, чтобы они поднимали ваш инстут в рейтинге лучших: побеждали на международных олимпиадах, писали лучшие научные работы, совершали новые открытия в науке.

Вы принимаете в свой институт по резюме, которое, как вы считаете, более полно отражет опыт абитуриента. Кому вы отдадите предпочтение: ученику из частной школы или из государственной?

В самом деле, существует некий стереотип: в частных школах качество обучения лучше, чем в государственных. Ведь частные школы берут деньги за индивидуальный подход к обучению, за, более высоквалифицированных и вовлечённых преподавателе, за более качественную инфраструктуру и общее оснащение.

В своей гипотезе, на примере имеющейся выборки, я хочу узнать: **Верно ли, что студенты, обучающиеся в частных школах, имеют более высокие шансы сдать экзамен успешно?**


**Гипотезы**

Н0: Нет разницы между средними баллами JAMB у студентов из государственных и частных школ.

Н1: Средний балл JAMB у студентов из частных школ ниже , чем у студентов из государственных.

In [23]:
from scipy import stats as st
# Срезы по типу школы
public = jamb[jamb['school_type'] == 'Public']['jamb_score']
private = jamb[jamb['school_type'] != 'Public']['jamb_score']

# Тест Шапиро-Уилка для проверки нормальности распределения
shapiro_public = st.shapiro(public)
print(f"Государственная школа: statistic = {shapiro_public.statistic:.4f}, p-value = {shapiro_public.pvalue:.2e}")

shapiro_private = st.shapiro(private)
print(f"Частная школа: statistic = {shapiro_private.statistic:.4f}, p-value = {shapiro_private.pvalue:.2e}")

alpha = 0.05

if shapiro_public.pvalue < alpha:
    print("Данные в группе с государственными школами не распределены нормально")
else:
    print("Данные в группе с государственными школами распределены нормально")

if shapiro_private.pvalue < alpha:
    print("Данные в группе с частными школами не распределены нормально")
else:
    print("Данные в группе с частными школами распределены нормально")


# Если хотя бы одна группа не нормальна — используем тест Манна-Уитни
result = st.mannwhitneyu(private, public, alternative='less')

print(f"\nU-статистика: {result.statistic:.2f}, p-значение: {result.pvalue:.2e}")

if result.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу')

Государственная школа: statistic = 0.9646, p-value = 2.36e-29
Частная школа: statistic = 0.9760, p-value = 1.17e-13
Данные в группе с государственными школами не распределены нормально
Данные в группе с частными школами не распределены нормально

U-статистика: 2629333.00, p-значение: 1.00e+00
Не получилось отвергнуть нулевую гипотезу


Сначала я использую тест Шапиро-Уилка — это статистический тест для проверки нормальности распределения . Он помогает определить, похожи ли данные на нормальное распределение. В данном случае - распределение баллов в частных и государственных школах.

Увидев, что распределение не нормально, использую тест Манна-Уитни  — это непараметрический тест , который используется для сравнения двух независимых групп , когда распределение не является нормальным. Вместо сравнения средних значений, как делает t-тест, тест Манна-Уитни сравнивает ранги.

Принцип работы:
Все значения из обеих групп объединяются.
Каждому значению присваивается ранг, где чем выше значение — тем выше ранг.
Вычисляется U-статистика , которая говорит о том, насколько сильно перекрываются распределения двух групп.

Мы не считаем средние , а просто смотрим, кто чаще получает более высокие баллы.

**Выводы:**
Анализ показал, что различия в результатах экзамена JAMB между студентами из частных и государственных школ не являются статистически значимыми . Несмотря на то, что выборка по частным школам представлена в достаточном количестве (25.3%), средние баллы этих студентов не превышают результаты студентов из государственных школ. Это может говорить о том, что тип образовательного учреждения не оказывает решающего влияния на итоговый результат при прочих равных условиях.